# Dask - Using Maxwell via dask_jobqueue 

### Utility function

In [ ]:
def print_squeue(username='cpassow'):
    import subprocess
    process = subprocess.Popen("squeue -u "+ username, stdout=subprocess.PIPE, shell=True)
    (output, err) = process.communicate()
    print('Total Jobs:', str(output).count(username))
    print('Running Jobs:', str(output).count(' R '))
    print('Pending Jobs:', str(output).count(' PD '))
    
print_squeue()

## Cluster setup

In [ ]:
from dask_jobqueue import SLURMCluster
from distributed import Client

In [ ]:
cluster = SLURMCluster(queue = 'ps', cores =30 , processes = 1, memory = '200GB', walltime = '00:30:00',
                      local_directory = './slurm_out', log_directory = './slurm_logs')

In [ ]:
cluster.scale(5)

In [ ]:
print_squeue()

In [ ]:
client = Client(cluster)

In [ ]:
client

## Problem: timing jumps in FL1 pp-laser

During a recent beamtime we observed the pp-laser jumps in timing and decided to record this behavior via a FLASH GHz ADC.

![title](jddd_adc.png)

### Imports 

In [ ]:
import numpy as np
import glob

from beamtimedaqaccess import BeamtimeDaqAccess 

### HDF files of 1 FEL block 

In [ ]:
root_dir = '/asap3/flash/gpfs/bl1/2019/data/11006508/raw/hdf/online-1/'
trace_addr = '/FL1/Experiment/BL1/ADQ412 GHz ADC/CH01/TD'

daq = BeamtimeDaqAccess.create(root_dir)

### Analysis function

In [ ]:
def get_rising_edges(run_number):
    trace = np.asarray(daq.allValuesOfRun(trace_addr, run_number))
    rising_edges = [np.argmax(trace[0][index] > 100) for index in range(len(trace[0])) if np.argmax(trace[0][index]) > 0 ]
    unique, counts = np.unique(rising_edges, return_counts=True)    
    edges = dict(zip(unique, counts))
    return edges

def counting(edges):
    late = np.sum((np.asarray([v for k,v in edges.items() if k >= 2100])))
    early = np.sum((np.asarray([v for k,v in edges.items() if k > 10 and k < 2100])))
    if early > 0 and late > 0:
        ratio = early / (early + late)
        ratio = float("{0:.2f}".format(ratio))
    else:
        ratio = np.nan
    return [late,early, ratio]

### Check # of DAQ runs in FEL block

In [ ]:
runs = [int(f[93:98]) for f in glob.glob(root_dir + '/fl1user1/*_file1_*')]
runs.sort()
print('total # of DAQ runs:',len(runs))

In [ ]:
runs = runs[0:3]

In [ ]:
import dask 

number_of_edges_per_run = []
for run in runs:
    number_of_edges = dask.delayed(get_rising_edges)(run)
    number_of_edges_per_run.append(number_of_edges)

In [ ]:
%%time

dict_early_late = dask.compute(*number_of_edges_per_run)

## Statistic for eLog

In [ ]:
results = [counting(dict_early_late[index]) for index in range(len(dict_early_late))]

for i in range(len(runs)):
    if len(dict_early_late[i]) > 1:
        print('Run:',runs[i],'Time Bins',dict_early_late[i])
    else:
        print('Run:',runs[i],'no Laser')
        
print('\nRUN | late | early | ratio')
for i in range(len(results)):
    print(runs[i], results[i])

## Cluster termination

In [ ]:
cluster.close()
client.close()

In [ ]:
print_squeue()